<a href="https://colab.research.google.com/github/tpmmthomas/Sepsis-diagnosis-from-pairwise-single-cell-RNA-Continued/blob/master/NEW_NEW_capsnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, optimizers
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D, BatchNormalization
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from tensorflow import keras
from sklearn.metrics import *
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from PIL import Image
from tensorflow.keras import initializers, layers
from datetime import date
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.multi_gpu_utils import multi_gpu_model
from tensorflow.python.client import device_lib
from trans import *
from cap import *

2024-06-24 21:05:20.815001: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
xtrain = r"../data/dataSC/NEW_training_sample.csv.gz"
ytrain = r"../data/dataSC/NEW_training_label.csv.gz"
xtest = r"../data/dataSC/NEW_testing_sample.csv.gz"
ytest = r"../data/dataSC/NEW_testing_label.csv.gz" 

i=0
samplesdf = pd.read_csv(xtrain,compression ="gzip",delimiter=',')
x_train = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytrain,compression ="gzip",delimiter=',')
y_train = samplesdf.to_numpy()

samplesdf = pd.read_csv(xtest,compression ="gzip",delimiter=',')
x_test = samplesdf.to_numpy()

samplesdf = pd.read_csv(ytest,compression ="gzip",delimiter=',')
y_test = samplesdf.to_numpy()

print("done")
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

done
(38483, 3030)
(38483, 1)
(4265, 3030)
(4265, 1)


In [4]:
common_indicator = pd.read_csv('../data/dataBulk/common_rna_indicator.csv',index_col=0).values.squeeze().tolist()
x_train = x_train.T[common_indicator].T
print(x_train.shape)
x_test = x_test.T[common_indicator].T
print(x_test.shape)

(38483, 2869)
(4265, 2869)


In [5]:
# train, validation, test ratio = 80%:10%:10%
xtr,xval, ytr,yval = train_test_split(x_train,y_train,test_size=4265,random_state=42)
print(xtr.shape,ytr.shape)
print(xval.shape,yval.shape)

(34218, 2869) (34218, 1)
(4265, 2869) (4265, 1)


In [6]:
xtr=tf.reshape(xtr,(len(xtr),2869,1))
xval=tf.reshape(xval,(len(xval),2869,1))
x_test=tf.reshape(x_test,(len(x_test),2869,1))
print(xtr.shape)
print(xval.shape)
print(x_test.shape)

2024-06-24 18:40:50.296483: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-06-24 18:40:50.299991: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-06-24 18:40:50.317434: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:17:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.68GiB deviceMemoryBandwidth: 715.34GiB/s
2024-06-24 18:40:50.317693: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:73:00.0 name: NVIDIA RTX A5000 computeCapability: 8.6
coreClock: 1.695GHz coreCount: 64 deviceMemorySize: 23.67GiB deviceMemoryBandwidth: 715.34GiB/s
2024-06-24 18:40:50.317721: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-06-24 

(34218, 2869, 1)
(4265, 2869, 1)
(4265, 2869, 1)


In [8]:
def CapsNet(input_shape, n_class, routings):
    inputs = layers.Input(shape=input_shape)
    primarycaps = PrimaryCap(inputs, dim_capsule=8, n_channels=1, kernel_size=1, strides=1, padding='valid')
    groupcaps,routing_weight = CapsuleLayer(num_capsule=n_class, dim_capsule=16, routings=routings,
                             name='groupcaps')(primarycaps)

    transformer_block = TransformerBlock(embed_dim=16, num_heads=2, ff_dim=128)
    x = transformer_block(groupcaps)
    x = layers.Flatten()(x)
    x = layers.Dropout(0.1)(x)
    x = layers.Dense(150, activation="relu")(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = models.Model(inputs, outputs)
    return model


def margin_loss(y_true, y_pred):
    L = y_true * K.square(K.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * K.square(K.maximum(0., y_pred - 0.1))
    return K.mean(K.sum(L, 1))


def test(model, X_test, y_test, args):
    y_pred, x_recon = model.predict(X_test, batch_size=20)
    print('-'*30 + 'Begin: test' + '-'*30)
    print('Test acc:', np.sum(np.argmax(y_pred, 1) == np.argmax(y_test, 1))/y_test.shape[0])
    return y_pred

In [9]:
 # define model
model= CapsNet(input_shape=xtr[1,:,:].shape,n_class=20,routings=5)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2869, 1)]         0         
_________________________________________________________________
weightlayer (Weightlayer)    ((None, 2869, 8), (1, 286 22952     
_________________________________________________________________
primarycap_reshape (Reshape) (None, 2869, 8)           0         
_________________________________________________________________
primarycap_squash (Lambda)   (None, 2869, 8)           0         
_________________________________________________________________
groupcaps (CapsuleLayer)     ((None, 20, 16), (None, 2 7344640   
_________________________________________________________________
transformer_block (Transform (None, 20, 16)            5392      
_________________________________________________________________
flatten (Flatten)            (None, 320)               0     

In [10]:
# train or test
# compile the model
model.compile(optimizer=optimizers.Adam(lr=0.0001, amsgrad=True),loss='binary_crossentropy', metrics=["accuracy"])

from tensorflow.keras import callbacks
earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", 
                                        mode ="min", patience = 5, 
                                        restore_best_weights = True)
history = model.fit(xtr, ytr, batch_size=32, epochs=20,validation_data=(xval, yval),
               shuffle=True)

today = date.today()
model.save('../modelsave/weightmodel')

2024-06-24 18:41:03.694630: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-06-24 18:41:03.695116: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2400000000 Hz


Epoch 1/20


2024-06-24 18:41:06.462005: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10


1070/1070 [==============================] - 194s 178ms/step - loss: 0.5883 - accuracy: 0.6653 - val_loss: 0.4464 - val_accuracy: 0.7848
Epoch 2/20
1070/1070 [==============================] - 194s 181ms/step - loss: 0.4066 - accuracy: 0.8127 - val_loss: 0.4148 - val_accuracy: 0.8082
Epoch 3/20
1070/1070 [==============================] - 194s 181ms/step - loss: 0.3695 - accuracy: 0.8301 - val_loss: 0.4050 - val_accuracy: 0.8148
Epoch 4/20
1070/1070 [==============================] - 193s 180ms/step - loss: 0.3484 - accuracy: 0.8388 - val_loss: 0.4088 - val_accuracy: 0.8185
Epoch 5/20
1070/1070 [==============================] - 192s 180ms/step - loss: 0.3140 - accuracy: 0.8565 - val_loss: 0.3626 - val_accuracy: 0.8347
Epoch 6/20
1070/1070 [==============================] - 193s 180ms/step - loss: 0.2670 - accuracy: 0.8805 - val_loss: 0.3527 - val_accuracy: 0.8511
Epoch 7/20
1070/1070 [==============================] - 192s 180ms/step - loss: 0.2266 - accuracy: 0.9017 - val_loss: 0.359

2024-06-24 19:45:18.977358: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../0623weightmodel_rount5/assets


INFO:tensorflow:Assets written to: ../0623weightmodel_rount5/assets


In [11]:
testresult=model.predict(x_test)
i = 0
correct = 0
for x in testresult:
    if x >=0.5 and y_test[i] == 1:
        correct = correct + 1
    elif x < 0.5 and y_test[i] == 0:
        correct = correct + 1
    i = i + 1
testacc = correct/i
print("Testing accuracy:",testacc)

fpr, tpr, _ = roc_curve(y_test,testresult)
roc_auc = auc(fpr,tpr)
print("AUROC = %.05f"% roc_auc)

precision, recall, _ = precision_recall_curve(y_test,testresult)
prc_auc = auc(recall,precision)
print("AUPRC = %.05f"% prc_auc)
ss = np.zeros((len(testresult)))
i = 0
for x in testresult:
    if x >= 0.5:
        ss[i] = 1
    else:
        ss[i] = 0
    i = i + 1
f1s = f1_score(y_test,ss)
print("f1_score = %.05f"% f1s)

Testing accuracy: 0.8604923798358733
AUROC = 0.93352
AUPRC = 0.93105
f1_score = 0.85491


In [12]:
# pd.DataFrame(testresult).to_csv('../capsnetweighted_rount5.csv',header=None, index=None)
# pd.DataFrame(y_test).to_csv('../capsnet_y_rount5.csv',header=None, index=None)